In [8]:
import keras
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv2D, MaxPooling2D, Conv1D, Lambda
from keras import backend as K
import pandas as pd
import numpy as np
import datetime
from DatasetPreprocessor import DatasetPreprocessor

In [12]:
con_win_size = 9
num_classes = 21
num_strings = 6
input_shape = (192, con_win_size, 1)

filename = '00_BN2-131-B_solo'
dataset = DatasetPreprocessor()

In [22]:
#наверное надо было в класс вынести, но пох
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, list_IDs, x_raw, y_raw, batch_size=128, shuffle=True, label_dim = (6,21), spec_repr="c", con_win_size=9):
        
        self.list_IDs = list_IDs
        self.x_raw = x_raw
        self.y_raw = y_raw
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.label_dim = label_dim
        self.spec_repr = spec_repr
        self.con_win_size = con_win_size
        self.halfwin = con_win_size // 2
        
        self.X_dim = (self.batch_size, 192, self.con_win_size, 1)
        self.y_dim = (self.batch_size, self.label_dim[0], self.label_dim[1])
                    
        self.on_epoch_end()

    #первые 3 метода дефолтные (на https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly тоже самое)   
    def __len__(self):
        return int(np.floor(float(len(self.list_IDs)) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        X, y = self.__data_generation(list_IDs_temp)
        
        return X, y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
        
    def __data_generation(self, list_IDs_temp):
        X = np.empty(self.X_dim)
        y = np.empty(self.y_dim)

        for i, ID in enumerate(list_IDs_temp):

            # у чела для всех 22мс кусков всех аудиофайлов записаны индексы в отдельном файле (data/spec_repr/id.csv)
            # тут чел получает полный нарезанный аудиофайл, к куску которого относится текущий индекс
            #data_dir = self.data_path + self.spec_repr + "/"
            #filename = "_".join(ID.split("_")[:-1]) + ".npz"
            #frame_idx = int(ID.split("_")[-1])
            
            # тут он к аудиофайлу добавляет края размером в 4 (полвина от 9) (хз причём тут ядро свёртки, оно в моделе 3x3)
            # когда он берёт кусок от файла по индексу, он берёт 9 кусоков
            #loaded = np.load(data_dir + filename)
            #full_x = np.pad(loaded["repr"], [(self.halfwin,self.halfwin), (0,0)], mode='constant')
            #sample_x = full_x[frame_idx : frame_idx + self.con_win_size]
            #X[i,] = np.expand_dims(np.swapaxes(sample_x, 0, 1), -1)

            #y[i,] = loaded["labels"][frame_idx]

            #вместо того, чтобы загружать из файла, один аудиофайл гружу в параметрах класса и беру из поля
            #предобрабатываю так же, как и чел
            #наверное, придётся в файлах сохранять аудио и лады, т.к. иначе будем грузить весь гигабайт музыки в память
            full_x = np.pad(self.x_raw, [(self.halfwin,self.halfwin), (0,0)], mode='constant')
            sample_x = full_x[ID : ID + self.con_win_size]
            X[i,] = np.expand_dims(np.swapaxes(sample_x, 0, 1), -1)

            y[i,] = self.y_raw[ID]

        return X, y

In [3]:
def softmax_by_string(t):
    string_sm = []
    for i in range(num_strings):
        string_sm.append(K.expand_dims(K.softmax(t[:,i,:]), axis=1))
    return K.concatenate(string_sm, axis=1)

def catcross_by_string(target, output):
    loss = 0
    for i in range(num_strings):
        loss += K.categorical_crossentropy(target[:,i,:], output[:,i,:])
    return loss

def avg_acc(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1)))

In [4]:
#тут ничё не менял
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                        activation='relu',
                        input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))   
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes * num_strings))
model.add(Reshape((num_strings, num_classes)))
model.add(Activation(softmax_by_string))

model.compile(loss=catcross_by_string,
                optimizer=keras.optimizers.Adadelta(),
                metrics=[avg_acc])

In [11]:
x_raw, y_raw = dataset.process_audiofile(filename)

f:\Repos\GuitarClassification\Desktop\AIModule\DatasetPreprocessor.py:94: FutureWarning: Pass orig_sr=44100, target_sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  data = librosa.resample(data, sr_original, sr_downs)


In [21]:
#делю айдишники на обучение и тест
train_ids = []
test_ids = []
ids = list(range(0,len(x_raw)))
for i in ids:
    result = np.random.choice(2, p=[.65,.35])
    if result == 1:
        test_ids.append(i)
    else:
        train_ids.append(i)

In [23]:
train_generator = DataGenerator(train_ids, x_raw, y_raw, shuffle=True)
test_generator = DataGenerator(test_ids, x_raw, y_raw, shuffle=True)

In [24]:
#вроде фит запустился, но одну эпоху для одного аудиофайла прогоняло 2 часа, после этого забил
#хз с многопоточкой проблема, с железом или я чёто не так сделал
model.fit_generator(generator=train_generator,
                    validation_data=None,
                    epochs=8,
                    verbose=1,
                    use_multiprocessing=True,
                    workers=9)

C:\Users\dimao\AppData\Local\Temp\ipykernel_2672\1306836756.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/8
